# 3. Model Evaluation

We are now familiar with our data, so the next logical step is to explore the space of algorithms that will eventually yield a good model for the task we are trying to solve.

Our goal in this notebook is not to develop state of the art solutions, but rather spotcheck several architectures in order to see which will be promoted to the next phase of the process, centered around model optimization. 

Without further ado, let's get to it!

## Transfer Learning

In computer vision is always a good idea to start leveraging the knowledge of pre-trained models. This is known as transfer learning. 

Keras comes with a nice set of models trained on ImageNet, which is great. This time we'll use the Keras interface that comes bundled with TensorFlow, instead of the standalone version.

## Spotchecking

In order to evaluate a broad range of possible algorithms, we need to implement some methods first. Let's start by creating a function to load our data.

### Data

Given that we are spotchecking deep neural networks, we need to preserve as much memory as possible. For that reason, we are going to generate batches on demand directly from disk, using `flow_from_directory`. 

This function expects all the images corresponding to a class to be stored inside a subfolder with the name of that class. For that reason, we must reaccomodate our directory structure.

We'll also set aside 10% of our data for validation purposes.

In [ ]:
import glob
import cv2
import os
from sklearn.utils import shuffle

destination_directory = './dataset'

def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

VALIDATION_PROPORTION = 0.1
vehicles_images_path = shuffle(glob.glob('data/vehicles/*/*.png'))
split_point = int(len(vehicles_images_path) * VALIDATION_PROPORTION)

for i, image_path in enumerate(vehicles_images_path):
    image = load_image(image_path)
    
    if i < split_point:
        destination_path = os.path.join(destination_directory, 'valid', 'vehicle', f'{i}.png')
    else:
        destination_path = os.path.join(destination_directory, 'train', 'vehicle', f'{i}.png')
    
    cv2.imwrite(destination_path, image)
    
non_vehicles_images_path = shuffle(glob.glob('data/non-vehicles/*/*.png'))
split_point = int(len(non_vehicles_images_path) * VALIDATION_PROPORTION)
for i, image_path in enumerate(non_vehicles_images_path):
    image = load_image(image_path)
    
    if i < split_point:
        destination_path = os.path.join(destination_directory, 'valid', 'non_vehicle', f'{i}.png')
    else:
        destination_path = os.path.join(destination_directory, 'train', 'non_vehicle', f'{i}.png')
    
    cv2.imwrite(destination_path, image)

We loaded the examples and their labels in the `X` and `y` variables, respectively. We decided to use the following mapping for the labels:

```
1 --> Vehicle.
0 --> Non-Vehicle.
```

### Models

Now, lets's procede to define a method to retrieve all the models we want to evaluate. 

The `get_models` method will return a `dict` of triplets, where the keys correspond to the name of the pretrained model we are using, and the values are triplets where the first element contain the input preprocessing function corresponding to that pre-trained model, the second element is a function to construct the model itself and the last one contains the input size for that model.

The `get_model_with_new_top` sub-function takes a base (pre-trained) model and attaches a fully connected network on top of it. It also freezes all the layers in the pre-trained model. Finally, it compiles the model to use `adam` as the optimizer.

In [1]:
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import gc

SEED = 314159

def get_models(models=None):
    # Takes a base, pretrained model, and attaches a new FCN on top of it.
    def get_model_with_new_top(base_model):
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        for layer in base_model.layers:
            layer.trainable = False
            
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
        
    if models is None:
        models = dict()
        
    models['mobilenet'] = {
        'preprocessing_function': applications.mobilenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['resnet50'] = {
        'preprocessing_function': applications.resnet50.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['inceptionV3'] = {
        'preprocessing_function': applications.inception_v3.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['xception'] = {
        'preprocessing_function': applications.xception.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    models['nasnet_large'] = {
        'preprocessing_function': applications.nasnet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))),
        'input_shape': (331, 331)
    }
    
    models['nasnet_mobile'] = {
        'preprocessing_function': applications.nasnet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet121'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet169'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet201'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['inception_resnet_v2'] = {
        'preprocessing_function': applications.inception_resnet_v2.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))),
        'input_shape': (299, 299)
    }
    
    models['vgg16'] = {
        'preprocessing_function': applications.vgg16.preprocess_input, 
        'model_constructor': lambda: get_model_with_new_top(applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    models['vgg19'] = {
        'preprocessing_function': applications.vgg19.preprocess_input, 
        'model_constructor': lambda: get_model_with_new_top(applications.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    return models

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def train_and_evaluate_models(models, epochs=5):
    for model_name, model_data in models.items():
        m = model_data['model_constructor']()
        train_data_generator = ImageDataGenerator(preprocessing_function=model_data['preprocessing_function']).flow_from_directory('./dataset/train', 
                                                                                                                                   target_size=model_data['input_shape'],
                                                                                                                                   batch_size=32,
                                                                                                                                   class_mode='binary')
        
        valid_data_generator = ImageDataGenerator(preprocessing_function=model_data['preprocessing_function']).flow_from_directory('./dataset/valid', 
                                                                                                                                   target_size=model_data['input_shape'],
                                                                                                                                   batch_size=32,
                                                                                                                                   class_mode='binary')
        step_size_train = train_data_generator.n // train_data_generator.batch_size

        print(f'Training {model_name}')
        history = m.fit_generator(generator=train_data_generator,
                                  steps_per_epoch=step_size_train,
                                  validation_data=valid_data_generator,
                                  validation_steps=(valid_data_generator.n // valid_data_generator.batch_size),
                                  epochs=epochs)

        print('Number of parameters: {:,}'.format(m.count_params()))
        print('---------------\n\n')

        del m
        del history
        K.clear_session()
        gc.collect()

In [3]:
models = get_models()
train_and_evaluate_models(models)

Instructions for updating:
Colocations handled automatically by placer.
Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training mobilenet
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
207/207 [==============================] - 27s 132ms/step - loss: 0.1239 - acc: 0.9540 - val_loss: 0.1482 - val_acc: 0.9467
Epoch 2/5
207/207 [==============================] - 25s 120ms/step - loss: 0.0554 - acc: 0.9791 - val_loss: 0.0569 - val_acc: 0.9822
Epoch 3/5
207/207 [==============================] - 24s 118ms/step - loss: 0.0510 - acc: 0.9800 - val_loss: 0.1439 - val_acc: 0.9399
Epoch 4/5
207/207 [==============================] - 24s 117ms/step - loss: 0.0396 - acc: 0.9848 - val_loss: 0.0527 - val_acc: 0.9809
Epoch 5/5
207/207 [==============================] - 24s 117ms/step - loss: 0.0564 - acc: 0.9801 - val_loss: 0.0580 - val_acc: 0.9822
Number of parameters: 3,885,249
---------------




/usr/lib/python3/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training resnet50
Epoch 1/5
207/207 [==============================] - 74s 356ms/step - loss: 0.0916 - acc: 0.9630 - val_loss: 0.0534 - val_acc: 0.9809
Epoch 2/5
207/207 [==============================] - 70s 338ms/step - loss: 0.0523 - acc: 0.9809 - val_loss: 0.0938 - val_acc: 0.9672
Epoch 3/5
207/207 [==============================] - 70s 337ms/step - loss: 0.0279 - acc: 0.9909 - val_loss: 0.0544 - val_acc: 0.9768
Epoch 4/5
207/207 [==============================] - 71s 342ms/step - loss: 0.0716 - acc: 0.9833 - val_loss: 0.2844 - val_acc: 0.9139
Epoch 5/5
207/207 [==============================] - 70s 336ms/step - loss: 0.0627 - acc: 0.9810 - val_loss: 0.0463 - val_acc: 0.9822
Number of parameters: 24,768,385
---------------


Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training inceptionV3
Epoch 1/5
207/207 [==============================] - 54s 262ms/step - loss